In [1]:
import numpy as np
import pandas as pd
import bokeh as bk
import random

from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
sns.set()  # set Seaborn defaults
plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn
from sklearn.cluster import KMeans  # for clustering

from bokeh.io import output_notebook, show, reset_output, curdoc
from bokeh.models import Slider, CustomJS, Select
from bokeh.plotting import figure
from bokeh.layouts import layout, column
from bokeh.models import (
    ColumnDataSource, Div,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange,
    ImageURL
)
from bokeh.palettes import BuPu
from bokeh.palettes import Colorblind8
output_notebook()

Loading BokehJS ...

In [2]:
Eyetracking_data = pd.read_csv('metro_data.csv', encoding = 'latin1', sep = ";")
Eyetracking_data.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [3]:
Antwerpen_S1 = Eyetracking_data[Eyetracking_data['StimuliName'] == '01_Antwerpen_S1.jpg']
Antwerpen_S1.head()

,Timestamp,StimuliName,FixationIndex,FixationDuration,MappedFixationPointX,MappedFixationPointY,user,description
0,2586,01_Antwerpen_S1.jpg,9,250,1151,458,p1,color
1,2836,01_Antwerpen_S1.jpg,10,150,1371,316,p1,color
2,2986,01_Antwerpen_S1.jpg,11,283,1342,287,p1,color
3,3269,01_Antwerpen_S1.jpg,12,433,762,303,p1,color
4,3702,01_Antwerpen_S1.jpg,13,183,624,297,p1,color


In [4]:
#replacing weird coding fails in stimuli names
Eyetracking_data['StimuliName'] = Eyetracking_data['StimuliName'].replace({'24_Z?rich_S1.jpg': '24_Zurich_S1.jpg', '24_Zrich_S1.jpg' : '24_Zurich_S1.jpg',
                                                                          '24_Zrich_S2.jpg' : '24_Zurich_S2.jpg', '24_Z?rich_S2.jpg' : '24_Zurich_S2.jpg',
                                                                          '24b_Z?rich_S1.jpg' : '24b_Zurich_S1.jpg', '24b_Zrich_S1.jpg' : '24b_Zurich_S1.jpg',
                                                                          '24b_Z?rich_S2.jpg' : '24b_Zurich_S2.jpg', '24b_Zrich_S2.jpg' : '24b_Zurich_S2.jpg', 
                                                                           
                                                                           '12_Br?ssel_S1.jpg' : '12_Brussel_S1.jpg', '12_Brssel_S1.jpg' : '12_Brussel_S1.jpg',
                                                                          '12_Br?ssel_S2.jpg' : '12_Brussel_S2.jpg', '12_Brssel_S2.jpg' : '12_Brussel_S2.jpg',
                                                                          '12b_Br?ssel_S1.jpg' : '12b_Brussel_S1.jpg', '12b_Brssel_S1.jpg' : '12b_Brussel_S1.jpg',
                                                                          '12b_Br?ssel_S2.jpg' : '12b_Brussel_S2.jpg', '12b_Brssel_S2.jpg' : '12b_Brussel_S2.jpg',
                                                                           
                                                                          '14_D?sseldorf_S1.jpg' : '14_Dusseldorf_S1.jpg', '14_Dsseldorf_S1.jpg' : '14_Dusseldorf_S1.jpg',
                                                                          '14_D?sseldorf_S1.jpg' : '14_Dusseldorf_S2.jpg', '14_Dsseldorf_S2.jpg' : '14_Dusseldorf_S2.jpg',
                                                                          '14b_D?sseldorf_S1.jpg' : '14b_Dusseldorf_S1.jpg', '14b_Dsseldorf_S1.jpg' : '14b_Dusseldorf_S1.jpg',
                                                                          '14b_D?sseldorf_S2.jpg' : '14b_Dusseldorf_S2.jpg', '14b_Dsseldorf_S2.jpg' : '14b_Dusseldorf_S2.jpg',
                                                                           
                                                                          '15_G?teborg_S1.jpg' : '15_Goteborg_S1.jpg', '15_Gteborg_S1.jpg' : '15_Goteborg_S1.jpg',
                                                                          '15_G?teborg_S2.jpg' : '15_Goteborg_S2.jpg', '15_Gteborg_S2.jpg' : '15_Goteborg_S2.jpg',
                                                                          '15b_G?teborg_S1.jpg' : '15b_Goteborg_S1.jpg', '15b_Gteborg_S1.jpg' : '15b_Goteborg_S1.jpg',
                                                                          '15b_G?teborg_S2.jpg' : '15b_Goteborg_S2.jpg', '15b_Gteborg_S2.jpg' : '15b_Goteborg_S2.jpg',
                                                                          
                                                                          '04_K?ln_S1.jpg' : '04_Koln_S1.jpg', '04_Kln_S1.jpg' : '04_Koln_S1.jpg',
                                                                          '04_K?ln_S2.jpg' : '04_Koln_S2.jpg', '04_Kln_S1.jpg' : '04_Koln_S2.jpg',
                                                                           '04b_K?ln_S1.jpg' : '04b_Koln_S1.jpg', '04b_Kln_S1.jpg' : '04b_Koln_S1.jpg',
                                                                          '04b_K?ln_S2.jpg' : '04b_Koln_S2.jpg', '04b_Kln_S2.jpg' : '04b_Koln_S2.jpg',})

In [5]:
#Grouping data per map
Antwerpen_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01_Antwerpen_S1.jpg']
Antwerpen_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01_Antwerpen_S2.jpg']

Antwerpen_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01b_Antwerpen_S1.jpg']
Antwerpen_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '01b_Antwerpen_S2.jpg']

Berlin_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02_Berlin_S1.jpg']
Berlin_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02_Berlin_S2.jpg']

Berlin_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02b_Berlin_S1.jpg']
Berlin_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '02b_Berlin_S2.jpg']

Bordeaux_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03_Bordeaux_S1.jpg']
Bordeaux_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03_Bordeaux_S2.jpg']

Bordeaux_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03b_Bordeaux_S1.jpg']
Bordeaux_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '03b_Bordeaux_S2.jpg']

Köln_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '04_Koln_S1.jpg']
Köln_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '04b_Koln_S1.jpg']
Köln_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '04b_Koln_S2.jpg']

Frankfurt_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05_Frankfurt_S1.jpg']
Frankfurt_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05_Frankfurt_S2.jpg']

Frankfurt_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05b_Frankfurt_S1.jpg']
Frankfurt_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '05b_Frankfurt_S2.jpg']

Hamburg_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06_Hamburg_S1.jpg']
Hamburg_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06_Hamburg_S2.jpg']

Hamburg_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06b_Hamburg_S1.jpg']
Hamburg_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '06b_Hamburg_S1.jpg']

Moskau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07_Moskau_S1.jpg']
Moskau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07_Moskau_S2.jpg']

Moskau_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07b_Moskau_S1.jpg']
Moskau_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '07b_Moskau_S2.jpg']

Riga_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08_Riga_S1.jpg']
Riga_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08_Riga_S1.jpg']

Riga_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08b_Riga_S1.jpg']
Riga_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '08b_Riga_S1.jpg']
Tokyo_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09_Tokyo_S1.jpg']
Tokyo_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09_Tokyo_S2.jpg']

Tokyo_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09b_Tokyo_S1.jpg']
Tokyo_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '09b_Tokyo_S2.jpg']

Barcelona_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10_Barcelona_S1.jpg']
Barcelona_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10_Barcelona_S2.jpg']

Barcelona_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10b_Barcelona_S1.jpg']
Barcelona_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '10b_Barcelona_S2.jpg']

Bologna_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11_Bologna_S1.jpg']
Bologna_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11_Bologna_S2.jpg']

Bologna_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11b_Bologna_S1.jpg']
Bologna_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '11b_Bologna_S2.jpg']

Brüssel_S1  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12_Brüssel_S1.jpg']
Brüssel_S2  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12_Brüssel_S2.jpg']

Brüssel_S1b  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12b_Brüssel_S1.jpg']
Brüssel_S2b  = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '12b_Brüssel_S2.jpg']

Budapest_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13_Budapest_S1.jpg']
Budapest_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13_Budapest_S2.jpg']

Budapest_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13b_Budapest_S1.jpg']
Budapest_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '13b_Budapest_S2.jpg']

Düsseldorf_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14_Dusseldorf_S1.jpg']
Düsseldorf_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14_Dusseldorf_S2.jpg']

Düsseldorf_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14b_Dusseldorf_S1.jpg']
Düsseldorf_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '14b_Dusseldorf_S2.jpg']
                                      
Göteborg_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15_Goteborg_S1.jpg']
Göteborg_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15_Goteborg_S2.jpg']
                                      
Göteborg_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15b_Göteborg_S1.jpg']
Göteborg_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '15b_Göteborg_S2.jpg'] 

Hong_Kong_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16_Hong_Kong_S1.jpg']
Hong_Kong_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16_Hong_Kong_S2.jpg'] 
                                      
Hong_Kong_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16b_Hong_Kong_S1.jpg']
Hong_Kong_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '16b_Hong_Kong_S2.jpg'] 
                                      
Krakau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17_Krakau_S1.jpg']
Krakau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17_Krakau_S2.jpg']
                                      
Krakau_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17b_Krakau_S1.jpg']
Krakau_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '17b_Krakau_S2.jpg']
                                      
Ljubljana_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S1.jpg']
Ljubljana_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S2.jpg']
                                      
Ljubljana_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S1.jpg']
Ljubljana_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '18_Ljubljana_S2.jpg']                                      

New_York_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19_New_York_S1.jpg']
New_York_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19_New_York_S2.jpg']

New_York_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19b_New_York_S1.jpg']
New_York_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '19b_New_York_S2.jpg']


Paris_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20_Paris_S1.jpg']
Paris_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20_Paris_S2.jpg']

Paris_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20b_Paris_S1.jpg']
Paris_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '20b_Paris_S2.jpg']

Pisa_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21_Pisa_S1.jpg']
Pisa_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21_Pisa_S2.jpg']

Pisa_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21b_Pisa_S1.jpg']
Pisa_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '21b_Pisa_S2.jpg']

Venedig_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22_Venedig_S1.jpg']
Venedig_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22_Venedig_S2.jpg']

Venedig_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22b_Venedig_S1.jpg']
Venedig_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '22b_Venedig_S2.jpg']

Warschau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23_Warschau_S1.jpg']
Warschau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23_Warschau_S2.jpg']

Warschau_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23b_Warschau_S1.jpg']
Warschau_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '23b_Warschau_S2.jpg']


Zürich_S1 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24_Zürich_S1.jpg']
Zürich_S2 = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24_Zürich_S2.jpg']

Zürich_S1b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24b_Zürich_S1.jpg']
Zürich_S2b = Eyetracking_data.loc[Eyetracking_data['StimuliName'] == '24b_Zürich_S2.jpg']

In [6]:
maps = Eyetracking_data['StimuliName'].unique().tolist()
maps.sort()

In [7]:
stimulimap = Select(title="Stimulimap", value=maps[6], options=maps)

In [8]:
def update():
    stimulimap_val = stimulimap.value
    selected = Eyetracking_data.copy()
    if (stimulimap_val != ""):
        selected = selected[selected['StimuliName'].str.contains(stimulimap_val)==True]

    x = np.array(selected['MappedFixationPointX'])
    y = np.array(selected['MappedFixationPointY'])
    bins = hexbin(x, y, size=20)
    
    return bins

In [9]:
#coordinates = update()
#x = coordinates[0]
#y = coordinates[1]
#x = np.array(Antwerpen_S1['MappedFixationPointX'])
#y = np.array(Antwerpen_S1['MappedFixationPointY'])

In [10]:
#bins = hexbin(x, y, size=20)
bins = update()
p = figure(title="Heatmap", tools="wheel_zoom,pan,reset",
           match_aspect=True)
p.grid.visible = False
#p.image_url(url=['01_Antwerpen_S1.jpg'], x=0, y=0, w=1650, h=1200)
p.hex_tile(q="q", r="r", size=20, line_color=None, source=bins,
           fill_color=linear_cmap('counts', 'Plasma256', 0, max(bins.counts)))
#p.circle(x, y, color="white", size=1) #maybe function so that user can choose to show this
p.y_range.flipped = True
p.add_tools(HoverTool(
    tooltips=[("count", "@counts")],
    mode="mouse", point_policy="follow_mouse"
))


In [11]:
show(p)

In [12]:
#layout execution
controls = [stimulimap]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

inputs = column(*controls, width=320, height=1000)
inputs.sizing_mode = "fixed"

l = layout([
    [inputs, p],
], sizing_mode="fixed")

bins = update()  # initial load of the data

#coordinates = update()
#x = coordinates[0]
#y = coordinates[1]

curdoc().add_root(l)
curdoc().title = "Heatmap"

In [13]:
show(l)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

